# CSV a txts

Vamos a generar los inserts para las tablas de Autosummit Perú SAC

Importar librerías

In [119]:
import pandas as pd
import numpy as np

Abrir CSV

In [120]:
ds_ASP = pd.read_csv('ASP.csv', sep=',')

Ejemplos

In [121]:
ds_ASP['Cliente']

0                (42206340) DANIELLA MARIA BOLAÑOS GAMERO
1                         (20100115663) PANDERO S.A. EAFC
2       (09468059) KATIA NATHALI DE LOAYZA WONG DE PAC...
3        (46472213) JHONATHAN MITCHELL ANTEZANA ESCALANTE
4                (42607724) KRISCIA ZULAY REATEGUI ZAMORA
                              ...                        
1253                                                  NaN
1254                                                  NaN
1255                                                  NaN
1256                                                  NaN
1257                                                  NaN
Name: Cliente, Length: 1258, dtype: object

<p>Filtrado de colores</p>

In [122]:
colors_rep = ds_ASP['Color']
colors = []

for c in colors_rep:
    if isinstance(c, str) and c.strip() != '' and c.lower() != 'nan':
        if c not in colors:
            colors.append(c)

colors = sorted(colors)

text_colors = '-- migrate:up\n\n'
id = 1

for c in colors:
    c_escaped = c.replace("'", "''")
    text_colors += f"INSERT INTO colors (id, nombre) VALUES ({id}, '{c_escaped}');\n"
    id += 1

text_colors += '\n-- migrate:down\nDELETE FROM colors;'
with open('inserts_colors.sql', 'w') as f:
    f.write(text_colors)


<p>Filtrado por Asesor </p>

In [123]:
import re
asesores_rep = ds_ASP['Asesor']
asesores = []

#En caso tenga nombres "raros" el asesor
PALABRAS_INVALIDAS = {
    "PDI", "EXHIBICION", "ATE", "CASO", "ENTREGA"
}

#Almacenar todo en diccionario - para evitar duplicados
asesores = {} 

def es_asesor_valido(texto):
    if not isinstance(texto, str):
        return False

    texto = texto.strip().upper()

    # Debe tener al menos un espacio (nombre + apellido)
    if " " not in texto:
        return False

    # No debe contener comas ni números
    if "," in texto or re.search(r"\d", texto):
        return False

    # No debe contener palabras inválidas
    for palabra in PALABRAS_INVALIDAS:
        if palabra in texto:
            return False

    # Solo letras y espacios
    if not re.match(r"^[A-ZÁÉÍÓÚÑ ]+$", texto):
        return False

    return True


for c in asesores_rep:
    if not es_asesor_valido(c):
        continue

    c = c.strip().upper()
    nombres, apellidos = c.split(" ", 1)

    key = (nombres, apellidos)
    asesores[key] = True

sql_asesores = "-- migrate:up\n\n"
id_asesor = 1

for nombres, apellidos in sorted(asesores.keys()):
    sql_asesores += (
        "INSERT INTO asesores (id, nombres, apellidos) "
        f"VALUES ({id_asesor}, '{nombres}', '{apellidos}');\n"
    )
    id_asesor += 1

sql_asesores += "\n-- migrate:down\nDELETE FROM asesores;\n"

with open("inserts_asesores.sql", "w", encoding="utf-8") as f:
    f.write(sql_asesores)


<h3>Descomposición en marca (nombre) y modelo (nombre, version, año)</h3>

<p> Considerar que Marca no está de forma explícita en el CSV. Solución: inferir y filtrar con diccionarios </p>

In [124]:
import re

# =========================
# 1. COLUMNA CSV
# =========================
modelos_rep = ds_ASP['Modelo']

# Diccionarios (evitan duplicados automáticamente)
marcas = {}      # { "FORD": 1 }
modelos = {}     # { (modelo_base, version, traccion, anio, motor, marca): True }

# =========================
# CATÁLOGOS CONTROLADOS
# =========================
MAPA_MARCAS = {
    "FORD": [
        "TERRITORY", "RANGER", "F-150", "MAVERICK",
        "EXPLORER", "ESCAPE", "BRONCO",
        "MUSTANG", "EXPEDITION"
    ],
    "CHERY": [
        "TIGGO", "ARRIZO", "M7", "HIMLA"
    ]
}

VERSIONES_VALIDAS = {
    "TITANIUM", "TREND", "XLS", "XLT", "XL", "LTD",
    "PLATINUM", "RAPTOR", "BADLANDS", "LARIAT",
    "ACTIVE", "ST", "PREMIUM", "PRO", "MAX",
    "BIG", "BEND", "TREMOR", "GT"
}

TRACCIONES = {"4X2", "4X4", "AWD", "4WD"}
DESCARTES = {"MT", "AT", "CVT", "DCT", "FHEV", "MHEV", "PHEV", "GLP", "GNV"}

# =========================
# FUNCIONES
# =========================

def detectar_marca(texto: str):
    texto = texto.upper()
    for marca, modelos in MAPA_MARCAS.items():
        for m in modelos:
            if m in texto:
                return marca
    return None


def extraer_anio(texto: str):
    match = re.search(r"(20\d{2})", texto)
    return int(match.group(1)) if match else None


def separar_modelo(texto: str):
    tokens = texto.upper().split()

    motor = None
    traccion = None
    usados = set()

    for t in tokens:
        # Motor (1.5L, 2.0T, etc.)
        if re.match(r"\d\.\d(T|L)?", t):
            motor = float(re.findall(r"\d\.\d", t)[0])
            usados.add(t)

        # Tracción
        elif t in TRACCIONES:
            traccion = t
            usados.add(t)

        # Tokens técnicos descartables
        elif t in DESCARTES:
            usados.add(t)

    # limpiar tokens ya usados
    limpio = [t for t in tokens if t not in usados]

    modelo_base = limpio[0]
    version_tokens = [t for t in limpio[1:] if t in VERSIONES_VALIDAS]
    version_modelo = " ".join(version_tokens)

    return modelo_base, version_modelo, traccion, motor

# =========================
# PROCESAMIENTO DEL CSV
# =========================

for fila in modelos_rep:
    if not isinstance(fila, str) or fila.strip() == "":
        continue

    fila = fila.strip().upper()

    marca = detectar_marca(fila)
    anio = extraer_anio(fila)

    if not marca or not anio:
        continue

    modelo_base, version, traccion, motor = separar_modelo(fila)

    if not all([modelo_base, version, traccion, motor]):
        continue

    # registrar marca (PK lógica)
    if marca not in marcas:
        marcas[marca] = len(marcas) + 1

    key = (modelo_base, version, traccion, anio, motor, marca)
    modelos[key] = True

# =========================
# SQL: MARCAS
# =========================

sql_marcas = "-- migrate:up\n\n"

for marca, id_marca in marcas.items():
    sql_marcas += (
        f"INSERT INTO marcas (id_marca, nombre) "
        f"VALUES ({id_marca}, '{marca}');\n"
    )

sql_marcas += "\n-- migrate:down\nDELETE FROM marcas;\n"

with open("inserts_marcas.sql", "w", encoding="utf-8") as f:
    f.write(sql_marcas)

# =========================
# SQL: MODELOS
# =========================

sql_modelos = "-- migrate:up\n\n"
id_modelo = 1

for (modelo_base, version, traccion, anio, motor, marca) in modelos.keys():
    marca_id = marcas[marca]

    sql_modelos += (
        "INSERT INTO modelos "
        "(id_modelo, modelo_base, version_modelo, traccion, anio, motor, marca_id) "
        f"VALUES ({id_modelo}, '{modelo_base}', '{version}', "
        f"'{traccion}', {anio}, {motor}, {marca_id});\n"
    )
    id_modelo += 1

sql_modelos += "\n-- migrate:down\nDELETE FROM modelos;\n"

with open("inserts_modelos.sql", "w", encoding="utf-8") as f:
    f.write(sql_modelos)


<p>Filtrado de GPS</p>
